In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
    V1beta1ModelSpec,
    V1beta1ModelFormat,
    V1beta1TFServingSpec,
    utils
)
from kubernetes import client 

In [ ]:
namespace = utils.get_default_target_namespace()
service_name="sklearn-python-student99"

api_version = constants.KSERVE_GROUP + '/' + constants.KSERVE_V1BETA1_VERSION
storage = 'gs://kfserving-examples/models/sklearn/1.0/model'
image = "kserve/sklearnserver:v0.9.0"
isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(name=service_name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       sklearn=(V1beta1SKLearnSpec(
                                           image = image,
                                            storage_uri = storage))))
)

In [ ]:
KServe = KServeClient()
KServe.create(isvc)

In [ ]:
KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

In [ ]:
import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

In [ ]:
sklearn_iris_input={
    "instances": [
      [6.8,  2.8,  4.8,  1.4],
      [6.0,  3.4,  4.5,  1.6]
    ]
}

In [ ]:
ver_info = '/v1/models/'
operaton = ':predict'
rsvc_url = isvc_url + ver_info + service_name + operaton

response = requests.post(rsvc_url, json=sklearn_iris_input)
print(response.text)